In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

df_train=pd.read_csv("/content/drive/MyDrive/uba/vision 2/train.csv")
print(df_train.shape)
print(df_train.columns)

train_label = np.array(df_train["label"])
df_train.drop("label", 1, inplace=True)
print(df_train.columns)

train_images=np.zeros((len(df_train), 1, 28, 28))
for index, row in df_train.iterrows():
    fila=0
    columna=0
    for i in range(784):
        # print(row["pixel"+str(i)])
        columna +=1
        if (i+1)%28 == 0 and i<782:
            fila+=1
            columna=0
            # print((i+1)/28)
        elif i>782:
            columna-=1
        # print(columna)
        train_images[index, 0, fila, columna] = row["pixel"+str(i)]
    # print(train_images[index,0])
    # break

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images/255, train_label, test_size=0.2, random_state=0)


(60000, 785)
Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)


In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import dataloader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import pandas as pd
from torchvision.io import read_image

import torch
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(0)

train_dataset = TensorDataset(torch.tensor(X_train),torch.tensor(y_train)) # create your datset
train_dataset.transform = transforms.Compose([
                transforms.transforms.GaussianBlur(3,1.2),
                transforms.RandomAdjustSharpness(0.8,0.2),
                transforms.RandomAutocontrast(0.2),
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0) # create your dataloader

test_dataset = TensorDataset(torch.tensor(X_test),torch.tensor(y_test)) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True, num_workers=0) # create your dataloader

In [9]:
! conda install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=9.2 -c pytorch
import torchvision.models as models

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
shufflenet.conv1[0]=nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
# # resnet18.avgpool = Identity()
shufflenet.fc=nn.Linear(in_features=1024, out_features=10, bias=True)
shufflenet

/bin/bash: conda: command not found


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [10]:
model = shufflenet

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0


In [11]:

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 56.59166666666667%
